# Demo: HybridRetriever


In [2]:
import os
from dotenv import load_dotenv

# load neo4j credentials (and openai api key in background)
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI', 'bolt://localhost:7687')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME', 'neo4j')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE', None)

In [7]:
import logging
logging.getLogger("neo4j.notifications").setLevel(logging.ERROR)

In [8]:
import neo4j

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), database=NEO4J_DATABASE)

In [ ]:
from neo4j_graphrag.retrievers import HybridRetriever
from neo4j_graphrag.embeddings import OpenAIEmbeddings

hybrid_retriever = HybridRetriever(
    driver,
    vector_index_name="text_embeddings",
    fulltext_index_name="chunk_text",
    embedder=OpenAIEmbeddings(),
    return_properties=["text"],
)

In [6]:
hybrid_res = hybrid_retriever.search(
    query_text="How is precision medicine applied to Lupus? provide in list format"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($vector_index_name, $top_k, $query_vector) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS vector_index_max_score UNWIND nodes AS n RETURN n.node AS node, (n.score / vector_index_max_score) AS score UNION CALL db.index.fulltext.queryNodes($fulltext_index_name, $query_text, {limit: $top_k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS ft_index_max_score UNWIND nodes AS n RETURN n.node AS node, (n.score / ft_index_max_score) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $top_k RETURN no

In [23]:
hybrid_res.items[0].content

"{'text': 'precise and systematic fashion as suggested here.\\nFuture care will involve molecular diagnostics throughout\\nthe patient timecourse to drive the least toxic combination\\nof therapies. Recent evidence suggests a paradigm shift is\\non the way but it is hard to predict how fast it will come.\\nDisclosure\\nThe authors report no con ﬂicts of interest in this work.\\nReferences\\n1. Lisnevskaia L, Murphy G, Isenberg DA. Systemic lupus\\nerythematosus. Lancet .2014 ;384:1878 –1888. doi:10.1016/S0140-\\n6736(14)60128'}"